                      BEGIN WITH IMPORTING THE GRANTS DATA USING PANDAS And CLEANING
                         

In [1]:
import pandas as pd

In [2]:
g = pd.read_csv('grants.csv', sep = ';', index_col = "Project Number",\
                na_values = ["Nicht zuteilbar - NA","NPO (Biblioth., Museen, Verwalt.) - NPO",\
                             "data not included in P3"]) 

In [3]:
## Firstly, we clean the data and drop unnecessary columns for the purpose of this exercise.

g.drop(['Responsible Applicant','Project Title','Project Title English',\
        'Funding Instrument','Funding Instrument Hierarchy',"Start Date",\
        'End Date','Discipline Number','Discipline Name','Discipline Name Hierarchy',\
        'Keywords'], axis = 1, inplace = True)

In [4]:
## Always good to have a quick look at the data
g.head(10)

,Institution,University,Approved Amount
Project Number,,,
1,NaN,NaN,11619.0
4,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.0
5,Kommission für das Corpus philosophorum medii ...,NaN,79732.0
6,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.0
7,Schweiz. Thesauruskommission,NaN,120042.0
8,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,53009.0
9,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,25403.0
10,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,47100.0
11,NaN,Université de Lausanne - LA,25814.0


In [5]:
## Dropping incomplete data i.e. with two null values 
g.dropna(thresh = 2, inplace = True)

In [6]:
## The (almost) 'useful' data as there are some fields with Amount missing which we shall drop later
g.shape

(49847, 3)

          STEP 1 : WE GET DATAFRAMES WHERE WE HAVE EXACTLY ONE ENTRY EITHER INSTITUTION OR UNIVERSITY

In [7]:
ins_only = pd.DataFrame(g.loc[g.University.isnull()])
ins_only.drop('University', axis = 1, inplace = True)
ins_only.Institution.value_counts().shape

(1742,)

In [8]:
ins_only.head()

,Institution,Approved Amount
Project Number,,
5,Kommission für das Corpus philosophorum medii ...,79732.0
7,Schweiz. Thesauruskommission,120042.0
17,Schweizerische Rechtsquellen c/o Universität Z...,862200.0
36,Schweizerische Gesellschaft für Volkskunde,225000.0
37,Kuratorium Carl J. Burckhardt,179124.0


In [9]:
uni_only = pd.DataFrame(g.loc[(g.University.notnull())])
uni_only.drop('Institution', axis = 1, inplace = True)
uni_only.University.value_counts().shape

(75,)

In [10]:
uni_only.head()

,University,Approved Amount
Project Number,,
4,Université de Genève - GE,41022.0
6,Universität Basel - BS,52627.0
8,Université de Fribourg - FR,53009.0
9,Université de Fribourg - FR,25403.0
10,Universität Zürich - ZH,47100.0


In [11]:
institutions = ins_only.Institution.value_counts().index.values
universities = uni_only.University.value_counts().index.values

In [12]:
import googlemaps
import json
encoder = json.JSONEncoder()
decoder = json.JSONDecoder()

In [13]:
## Google Places API key 
client = googlemaps.Client(key = open('Mappyman.txt').read())

In [14]:
import numpy

                                     PUT YOUR GOOGLE API KEY in 'Mappyman.txt' to RUN

In [15]:
institutions_cantons = {}
universities_cantons = {}

In [16]:
## Script to download the canton corresponding to the institution, default is nan
for i in institutions:
    institutions_cantons[i] = numpy.nan
    response = (encoder.encode(client.places(i.decode('utf-8'))))
    if("Switzerland" in response):
        place_id = decoder.decode(response)['results'][0]['place_id']
        place_resp = client.place(place_id)
        canton_id = (item for item in place_resp['result']['address_components']if (item['types'] == [u'administrative_area_level_1', u'political'])).next()['short_name']
        institutions_cantons[i] = canton_id  

In [18]:
## Dumping file 
import json
json.dump(institutions_cantons, open("institutions_cantons.txt",'w'))

In [19]:
## Script to download the canton corresponding to the university, default is nan

for u in universities:
    universities_cantons[u] = numpy.nan
    resp = encoder.encode(client.places(u.decode('utf-8')))
    if "Switzerland" in resp:
        place_id = decoder.decode(resp)['results'][0]['place_id']
        place_resp = client.place(place_id)
        canton_id = (item for item in place_resp['result']['address_components']\
                     if (item['types'] == [u'administrative_area_level_1', u'political'])).next()['short_name']
        universities_cantons[u] = canton_id

In [20]:
## Dumping file
json.dump(universities_cantons, open("universities_cantons.txt",'w'))

                          CAN RUN THE FOLLOWING USING THE DUMPED JSON FILES

In [35]:
## We now map the values of our dictionary into the respective dataframes

ins_only.replace({"Institution": institutions_cantons }, inplace = True)
uni_only.replace({"University" : universities_cantons }, inplace = True)

//anaconda/envs/python2/lib/python2.7/site-packages/pandas/core/internals.py:4583: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  result = op(a, b)


In [36]:
## We drop the values that are nan i.e. not found in the google places API
institute = ins_only.dropna(how = 'any').reset_index()
university = uni_only.dropna(how = 'any').reset_index()

In [37]:
ins_groups =  institute.groupby("Institution").sum()
ins_groups = ins_groups.reset_index()
ins_groups.columns = ["Canton","Project Number","Amount" ]


uni_groups =  university.groupby("University").sum()
uni_groups = uni_groups.reset_index()
uni_groups.columns = ["Canton","Project Number","Amount" ]



In [56]:
final = pd.merge(ins_groups, uni_groups, on = "Canton", how = "outer")
final.fillna(0, inplace = True)


In [57]:
final['Total_amount'] = final['Amount_x'] + final['Amount_y']

In [58]:
final.drop(['Project Number_x','Amount_x','Project Number_y','Amount_y'], axis = 1 , inplace = True)

In [59]:
## Amounts in hundred thousands
final.Total_amount = final.Total_amount/100000

In [60]:
## Adding missing cantons for mapping topojson to the final dataframe
missing_cantons = pd.DataFrame([["AI",0.0], ["GL", 0.0],["NW", 0.0],[u'OW', 0.0]], columns=["Canton", "Total_amount"])

In [196]:
final_DF=final.append(missing_cantons, ignore_index = True)

,Canton,Total_amount
0,AG,1222.573649
1,AR,3.560000
2,BE,15833.923956
3,BL,55.792610
4,BS,14203.106890
5,FR,4628.999940
6,GE,19034.367161
7,GR,343.159720
8,JU,349.023585
9,LU,620.616633


                   WE HAVE NOW TO MAP THE VALUES ACCORDING TO THE AMOUNTS PER CANTON USING FOLIUM 

In [105]:
import folium

In [123]:
## Run with the topojson provided in the git directory 

topology_ch_cantons = r'ch-cantons.topojson.json'

map_cantons = folium.Map(location=[46.5, 8], zoom_start=7)

map_cantons.choropleth(geo_path = topology_ch_cantons,
            data=final_DF,
            columns=['Canton', 'Total_amount'],
            key_on='feature.id',
            threshold_scale=[0,100, 500, 1000, 1500, 2000],
            fill_color='PuRd',
            fill_opacity=0.5,
            line_opacity=0.8,
            legend_name='Total Amount In Hundreds Of Thousands CHF ',
            topojson='objects.cantons')

map_cantons

## Saving the map to HTML format for students' evaluation without calling the API

#map_cantons.save('map_cantons_final.html')

BONUS

In [167]:
french = final_DF.loc[final_DF['Canton'].isin(['VD','FR','NE','GE','JU','BE','VL'])]
italian = final_DF.loc[final_DF['Canton'].isin(['TI'])]
german = final_DF.loc[final_DF['Canton'].isin(['AG','BL','BS','GR','LU','SG','SH','SO','SZ','TG','UR','VS','ZG','ZH'])]
Rostigraben = pd.DataFrame([french['Total_amount'].sum(),italian['Total_amount'].sum(),german['Total_amount'].sum()],columns=["Total_amount"])
#Rostigraben.reset_index(level=1,inplace=True)
#Rostigraben.columns=["Part of Switzerland","Total"]
Rostigraben

,Total_amount
0,67511.604584
1,1211.653039
2,54648.543035


French part has the most funding, German part is closely behind, but the relative size of these parts and the assumptions we made about regional splits should be taken into account.